Requirements.txt

torch 2.0.0+cu117
transformers 4.28.1
numpy
typing
tqdm
gc
pandas
collections
sklearn
contextlib
io
IPython
os

In [1]:
%load_ext autoreload
%autoreload 2


import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3,4"
import sys
import torch
import pandas as pd
from torch import nn
import numpy as np
from torch.optim.lr_scheduler import ExponentialLR
import wandb

sys.path.append("../NLP-DL-Project-hypo-to-hyper/pipeline_src/")


from config.config import TaskConfig
from train import CustomScheduler, train
from logger.logger import WanDBWriter
from trainer.train_epoch import train_epoch, predict
from metrics.metrics import get_all_metrics
from dataset.dataset import init_data
from logger.logger import WanDBWriter





if torch.cuda.is_available():
    device='cuda'
    print('GPU')
else:
    device='cpu'
    print('CPU')
    
    
SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
print(torch.cuda.device_count())

GPU
3


In [2]:
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    AutoConfig,
    AutoModelForCausalLM,
    LlamaTokenizer,
    LlamaForCausalLM

)

from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict
    )


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /srv/home/rabikov/taxonomy_env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /srv/home/rabikov/taxonomy_env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/srv/home/rabikov/taxonomy_env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('FILE')}
  warn(msg)
/srv/home/rabikov/taxonomy_env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/srv/home/rabikov/taxonomy_env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/srv/home/rabikov/taxonomy_env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib64')

In [3]:
config = TaskConfig()

config.n_epochs = 4
config.batch_size = 16
config.lr = 3e-4
config.min_lr = 3e-6

config.validation = 1
config.save_every = 1
config.compute_metrics_every = 1


config.data_path = "SemEval2018-Task9/training/data/1A.english.training.data.txt"
config.gold_path = "SemEval2018-Task9/training/gold/1A.english.training.gold.txt"
config.test_data_path = "SemEval2018-Task9/test/data/1A.english.test.data.txt"
config.test_gold_path = "SemEval2018-Task9/test/gold/1A.english.test.gold.txt"

config.device = device
config.using_peft = False
config.model_type = 'Auto' #Auto or Llama
config.wandb_log_dir = '/raid/rabikov/wandb/'
config.model_checkpoint = 'EleutherAI/gpt-neo-125m'
config.exp_name = config.model_checkpoint.replace('/', '-')
config.saving_path = '/raid/rabikov/model_checkpoints/' + config.exp_name


In [4]:
if config.model_type == 'Auto':
    model_type = AutoModelForCausalLM
    tokenizer_type = AutoTokenizer
elif config.model_type == "Llama":
    model_type = LlamaForCausalLM
    tokenizer_type = LlamaTokenizer

model = model_type.from_pretrained(config.model_checkpoint,
    #load_in_8bit=True,
   # torch_dtype=torch.float16,
    device_map="auto",)

tokenizer = tokenizer_type.from_pretrained(
    config.model_checkpoint,
    padding_side="left",
)


In [5]:
if config.using_peft:
    LORA_R = 8
    LORA_ALPHA = 16
    LORA_DROPOUT= 0.05
    LORA_TARGET_MODULES = [
        "q",
        "v",
    ]
    
    #model = prepare_model_for_int8_training(model)
    config_lora = LoraConfig(
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
    # target_modules=LORA_TARGET_MODULES,
        lora_dropout=LORA_DROPOUT,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, config_lora)
    model.print_trainable_parameters()


In [6]:
train_dataset, test_dataset, train_loader, val_loader = init_data(tokenizer, config)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(
    model.parameters(), lr=config.lr, betas=(0.9, 0.98), eps=1e-9
)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=config.lr,
#                                                  steps_per_epoch=len(train_loader), epochs=config.n_epochs)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader) * config.n_epochs,
                                                        eta_min=config.min_lr)


In [8]:
# wandb
logger = WanDBWriter(config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vityavitalich. Use `wandb login --relogin` to force relogin


In [9]:
train(
        model,
        tokenizer,
        train_loader,
        val_loader,
        optimizer,
        scheduler,
        criterion,
        logger,
        config,
    )

Start of the epoch 0


eval going: 100%|██████████| 93/93 [00:23<00:00,  3.89it/s]


Start of the epoch 1


eval going: 100%|██████████| 93/93 [00:22<00:00,  4.07it/s]


Start of the epoch 2


eval going: 100%|██████████| 93/93 [00:26<00:00,  3.54it/s]


Start of the epoch 3


eval going: 100%|██████████| 93/93 [00:23<00:00,  3.93it/s]
